In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import math
rng = np.random.default_rng(42)

In [2]:
#from ipywidgets import interact, interactive
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import ipywidgets as widgets
import asyncio
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
import plotly
import math
import time

In [3]:
%%capture
%matplotlib ipympl

In [4]:
%%capture
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 180  
plt.ioff()

In [5]:
# -----------------------
# Geometry / cartoon knobs
# -----------------------
phi0 = 7*np.pi/6               # initial direction (rad)
R_VTX = [2.5, 5.0, 13.0, 18.0]   # VTX layers (cm)
R_EMCAL = 45.0                    # outer EMCal radius (cm), cartoon value
R0 = 55.0                         # electron curvature radius BEFORE emission (cm)
R1 = 30.0                         # AFTER emission (cm)  (smaller radius = tighter curve)
C0 = np.array([ R0*np.cos(phi0-0.5*np.pi), R0*np.sin(phi0-0.5*np.pi) ])       # pre-emission circle center (gives nice arc across origin)
charge_sign = +1                  # cartoon handedness (+1 = center on "left" of direction)
emit_layer_idx = 2                # index in R_VTX where emission happens (0..3). Here: 13 cm layer
n_hits_per_layer = 1             # number of "hits" to scatter on each layer for the cartoon
phi_window = (3*np.pi/4, 3*np.pi/2)  # angular window for drawing arcs (keeps picture compact)
phi_track = (-2*np.pi, -1*np.pi)

# -------------

In [6]:
# -------------
# Helper math
# -------------
def circle_intersections(C, R, r):
    """
    Intersections between a circle centered at C with radius R,
    and a concentric circle centered at (0,0) with radius r.
    Returns up to 2 points (x,y) as a list.
    """
    x0, y0 = C
    d = np.hypot(x0, y0)
    # Handle special/degenerate cases lightly (cartoon):
    if d == 0:  # concentric circles
        return []
    if d > R + r or d < abs(R - r):
        return []
    # a = distance along line C->origin from C to the chord midpoint
    a = (R**2 - r**2 + d**2) / (2*d)
    h2 = R**2 - a**2
    if h2 < 0: 
        h2 = 0
    h = math.sqrt(h2)
    # point P2 is along the line from C to 0 at distance a from C
    x2 = x0 + a * (-x0) / d
    y2 = y0 + a * (-y0) / d
    # offset vector perpendicular to (0 - C)
    rx = -y0 * (h / d)
    ry =  x0 * (h / d)
    return [(x2 + rx, y2 + ry), (x2 - rx, y2 - ry)]

In [7]:
def unit(v):
    n = np.linalg.norm(v)
    if n == 0: return v
    return v / n

def rot90(v):  # rotate +90°
    return np.array([-v[1], v[0]])

def arc_points(C, R, phis):
    return np.column_stack([C[0] + R*np.cos(phis), C[1] + R*np.sin(phis)])

In [8]:
def pick_intersection_on_window(points, phi_min, phi_max):
    """Pick the intersection whose polar angle (about origin) lies inside [phi_min, phi_max]."""
    for p in points:
        phi = math.atan2(p[1], p[0])
        if phi<0: phi += 2*math.pi
        print(f"  intersection at {p}, phi={phi:.2f} rad for points {points}")
        print(f"    checking window {phi_min:.2f} .. {phi_max:.2f}")
        if phi_min <= phi <= phi_max:
            print(f"  -> selected this one {p}")
            return np.array(p)
    # fallback: closest phi to window
    if not points: 
        return None
    best = min(points, key=lambda q: min( abs(math.atan2(q[1], q[0]) - phi_min),
                                          abs(math.atan2(q[1], q[0]) - phi_max)))
    return np.array(best)

In [9]:
# -----------------------------------
# Find emission point and post circle
# -----------------------------------
# Intersect pre-emission circle with the chosen VTX layer radius
emit_r = R_VTX[emit_layer_idx]
emit_pts = circle_intersections(C0, R0, emit_r)
P_emit = pick_intersection_on_window(emit_pts, *phi_window)
if P_emit is None:
    # If geometry is unlucky, nudge center a bit
    C0 = np.array([-(R0-3.0), 0.0])
    emit_pts = circle_intersections(C0, R0, emit_r)
    P_emit = pick_intersection_on_window(emit_pts, *phi_window)

# Tangent direction at emission (perp to radius vector from C0 to P_emit)
rhat = unit(P_emit - C0)
that = rot90(rhat)               # +90° rotation gives forward direction for this handedness
nhat = rot90(that)               # normal pointing to circle center side
# Post-emission circle center so motion is C1 with same tangent at P_emit
C1 = P_emit + charge_sign * R1 * nhat

# Compute EMCal intersections for pre and post circles
emcal_pre_pts  = circle_intersections(C0, R0, R_EMCAL)
emcal_post_pts = circle_intersections(C1, R1, R_EMCAL)
P_emcal_pre  = pick_intersection_on_window(emcal_pre_pts,  *phi_window)
P_emcal_post = pick_intersection_on_window(emcal_post_pts, *phi_window)

  intersection at (-11.947613381666825, -5.123917884023518), phi=3.55 rad for points [(-11.947613381666825, -5.123917884023518), (10.411249745303186, 7.784977761106608)]
    checking window 2.36 .. 4.71
  -> selected this one (-11.947613381666825, -5.123917884023518)
  intersection at (-44.7654654219144, -4.588366327941298), phi=3.24 rad for points [(-44.7654654219144, -4.588366327941298), (26.356374512823503, 36.47384710364109)]
    checking window 2.36 .. 4.71
  -> selected this one (-44.7654654219144, -4.588366327941298)
  intersection at (-44.612066720294436, 5.896058254800234), phi=3.01 rad for points [(-44.612066720294436, 5.896058254800234), (0.6512891330810611, 44.9952866694405)]
    checking window 2.36 .. 4.71
  -> selected this one (-44.612066720294436, 5.896058254800234)


In [ ]:
# -----------------------------------
# "Histogram on a circle" mini-utility
# -----------------------------------
def draw_polar_hist(ax, radius, angles, weights=None, bar_width=0.04, thickness=0.6, **kwargs):
    """
    Draws tiny radial bars centered on given angles at a given radius.
    thickness is bar radial thickness (cm); bar_width in radians.
    - If kwargs has 'alpha', it's treated as a global scale and multiplied
      into the per-bar alpha based on weights.
    """
    # Pull any externally provided alpha out of kwargs so we don't pass two alphas to ax.fill
    alpha_scale = kwargs.pop('alpha', 1.0)

    if weights is None:
        weights = np.ones_like(angles)
    weights = np.asarray(weights)
    if weights.size == 0:
        return
    maxw = np.max(weights)
    if maxw <= 0:
        maxw = 1.0

    for ang, w in zip(angles, weights):
        r0 = radius - thickness*0.5
        r1 = radius + thickness*0.5
        xs = [r0*np.cos(ang-bar_width/2), r0*np.cos(ang+bar_width/2),
              r1*np.cos(ang+bar_width/2), r1*np.cos(ang-bar_width/2)]
        ys = [r0*np.sin(ang-bar_width/2), r0*np.sin(ang+bar_width/2),
              r1*np.sin(ang+bar_width/2), r1*np.sin(ang-bar_width/2)]
        # per-bar alpha: map weights to [0.25, 1.0], then scale by alpha_scale
        bar_alpha = min(0.25 + 0.75*(w/maxw), 1.0) * alpha_scale
        ax.fill(xs, ys, alpha=bar_alpha, **kwargs)


# Fake residual-like φ histograms for the cartoon (replace with your real ones)
def fake_phi_hist(center_phi, spread=0.2, n=24):
    phis = rng.normal(center_phi, spread, size=n)
    # wrap to [-pi, pi]
    phis = ( (phis + np.pi) % (2*np.pi) ) - np.pi
    w = 1.0 + 2.5*rng.random(n)
    return phis, w

In [ ]:
# Choose a central φ for each layer around the track direction there
phi_emit = math.atan2(P_emit[1], P_emit[0])
layer_phi_centers = []
for r in R_VTX:
    # find intersections to estimate the local crossing angle ≈ arc φ where it hits the layer
    pts = circle_intersections(C0, R0, r)
    pt  = pick_intersection_on_window(pts, *phi_window)
    if pt is None:
        layer_phi_centers.append(phi_emit)
    else:
        layer_phi_centers.append(math.atan2(pt[1], pt[0]))

layer_hists = [fake_phi_hist(cp, spread=0.18, n=20) for cp in layer_phi_centers]

# -----------------------------------
# Animation
# -----------------------------------
fig, ax = plt.subplots(figsize=(7,7))
ax.set_aspect('equal', adjustable='box')
ax.set_xlim(-100, 30)
ax.set_ylim(-70, 70)
ax.set_xlabel("x (cm)")
ax.set_ylabel("y (cm)")
title = ax.set_title("Bremsstrahlung cartoon: fit before/after emission, γ to EMCal")

# Precompute arcs (parameterized by φ about the circle centers)
phi_arc = np.linspace(phi_window[0], phi_window[1], 600)

pre_arc  = arc_points(C0, R0, phi_arc)
post_arc = arc_points(C1, R1, phi_arc)

# Helper to scatter cartoon "hits" on each VTX layer near the crossing
def layer_hits(radius, center_phi, n=n_hits_per_layer, jitter=0.03):
    phis = rng.normal(center_phi, 0.035, size=n)
    rs   = rng.normal(radius,     jitter, size=n)
    xs = rs*np.cos(phis); ys = rs*np.sin(phis)
    return xs, ys

# Pre-build hit clouds
layer_hit_points = []
for r, (phis, w) in zip(R_VTX, layer_hists):
    cp = np.mean(phis)  # rough center
    xs, ys = layer_hits(r, cp)
    layer_hit_points.append((xs, ys))

# Make a single deviating hit on the emission layer to indicate the “off-circle” measurement
dev_idx = emit_layer_idx
dev_x = P_emit[0] + 0.9*rot90(unit(P_emit - C0))[0]   # a bit along the tangent direction
dev_y = P_emit[1] + 0.9*rot90(unit(P_emit - C0))[1]

# For stepwise reveal
total_frames = 14

  intersection at (-2.19291336918183, -1.2004711388714717), phi=3.64 rad for points [(-2.19291336918183, -1.2004711388714717), (2.13609518736365, 1.2988831165742531)]
    checking window 2.36 .. 4.71
  -> selected this one (-2.19291336918183, -1.2004711388714717)
  intersection at (-4.439287797880528, -2.3005920645758042), phi=3.62 rad for points [(-4.439287797880528, -2.3005920645758042), (4.2120150706078086, 2.6942399753869157)]
    checking window 2.36 .. 4.71
  -> selected this one (-4.439287797880528, -2.3005920645758042)
  intersection at (-11.947613381666825, -5.123917884023518), phi=3.55 rad for points [(-11.947613381666825, -5.123917884023518), (10.411249745303186, 7.784977761106608)]
    checking window 2.36 .. 4.71
  -> selected this one (-11.947613381666825, -5.123917884023518)
  intersection at (-16.851063435670774, -6.327848061308007), phi=3.50 rad for points [(-16.851063435670774, -6.327848061308007), (13.905608890216238, 11.429524985419963)]
    checking window 2.36 .. 

In [ ]:
def draw_static():
    # VTX layers
    for r in R_VTX:
        th = np.linspace(0, 2*np.pi, 400)
        ax.plot(r*np.cos(th), r*np.sin(th), lw=2)

    # EMCal ring
    th = np.linspace(0, 2*np.pi, 600)
    ax.plot(R_EMCAL*np.cos(th), R_EMCAL*np.sin(th), lw=2, ls='--')

def draw_histograms(alpha_scale=1.0):
    for (r, (phis, w)) in zip(R_VTX, layer_hists):
        draw_polar_hist(ax, r, phis, weights=w, bar_width=0.08, thickness=0.7, color='C1', alpha=0.35*alpha_scale)

def draw_hits(show_deviant=False):
    for i, (r, pts) in enumerate(zip(R_VTX, layer_hit_points)):
        xs, ys = pts
        ax.scatter(xs, ys, s=12, alpha=0.7)
        if show_deviant and i == dev_idx:
            ax.scatter([dev_x], [dev_y], s=40, marker='X')
            
def draw_pre_fit(up_to_P=None):
    ax.plot(pre_arc[:,0], pre_arc[:,1], lw=3, label='fit before emission')

def draw_post_fit(from_P=None):
    ax.plot(post_arc[:,0], post_arc[:,1], lw=3, label='fit after emission')


# --- at emission point P_emit, compute tangent of the PRE track ---
# If you already had a pre circle center (C0_old) you can get t_hat from it:
rhat = (P_emit - C0) / np.linalg.norm(P_emit - C0)   # radius direction (center -> point)
t_hat = np.array([-rhat[1], rhat[0]])                # CCW tangent; flip sign if motion is opposite

# Choose which side the curvature lives on (sign of q*Bz):
charge_sign = +1   # try +1; if arc bends the wrong way, use -1

# Rebuild BOTH centers to enforce tangent continuity at emission
C0 = P_emit + charge_sign * R0 * np.array([-t_hat[1], t_hat[0]])   # n_hat = rot90(t_hat)
C1 = P_emit + charge_sign * R1 * np.array([-t_hat[1], t_hat[0]])

# Recompute EMCal intersection points with the corrected centers
emcal_pre_pts  = circle_intersections(C0, R0, R_EMCAL)
emcal_post_pts = circle_intersections(C1, R1, R_EMCAL)
P_emcal_pre  = pick_intersection_on_window(emcal_pre_pts,  *phi_window)
P_emcal_post = pick_intersection_on_window(emcal_post_pts, *phi_window)

def draw_gamma():
    # unit tangent at emission (you already have t_hat)
    v = t_hat / np.linalg.norm(t_hat)

    # Ensure the photon goes OUTWARD (|x| increasing). If not, flip v.
    # d|P_emit + s v|/ds at s=0 = (P_emit · v)/|P_emit|
    if np.dot(P_emit, v) < 0.0:
        v = -v

    # Solve |P_emit + s v| = R_EMCAL  for s >= 0
    Pv = float(np.dot(P_emit, v))
    P2 = float(np.dot(P_emit, P_emit))
    disc = Pv*Pv - (P2 - R_EMCAL*R_EMCAL)
    if disc < 0:
        # No intersection (numerical/geometry issue); bail gracefully
        return

    # With v chosen outward, this root is non-negative
    s = -Pv + math.sqrt(disc)

    Q = P_emit + s*v
    ax.plot([P_emit[0], Q[0]], [P_emit[1], Q[1]], ls='--')



def draw_phi_extrap():
    if P_emcal_post is not None:
        # φ at EMCal after emission (final fit)
        phi_post = math.atan2(P_emcal_post[1], P_emcal_post[0])
        ax.plot([0, R_EMCAL*np.cos(phi_post)], [0, R_EMCAL*np.sin(phi_post)], ls=':', lw=2)
        ax.annotate(r"$\phi_{\mathrm{fit,\,post}}$", 
                    xy=(R_EMCAL*np.cos(phi_post), R_EMCAL*np.sin(phi_post)),
                    xytext=(R_EMCAL*np.cos(phi_post)+5, R_EMCAL*np.sin(phi_post)),
                    arrowprops=dict(arrowstyle="->"))

    if P_emcal_pre is not None:
        # φ at EMCal if no brem (for contrast)
        phi_pre = math.atan2(P_emcal_pre[1], P_emcal_pre[0])
        ax.plot([0, R_EMCAL*np.cos(phi_pre)], [0, R_EMCAL*np.sin(phi_pre)], ls='--', lw=1)
        ax.annotate(r"$\phi_{\mathrm{fit,\,pre}}$", 
                    xy=(R_EMCAL*np.cos(phi_pre), R_EMCAL*np.sin(phi_pre)),
                    xytext=(R_EMCAL*np.cos(phi_pre)+5, R_EMCAL*np.sin(phi_pre)-6),
                    arrowprops=dict(arrowstyle="->"))

  intersection at (-44.7654654219144, -4.588366327941305), phi=3.24 rad for points [(-44.7654654219144, -4.588366327941305), (26.356374512823503, 36.47384710364109)]
    checking window 2.36 .. 4.71
  -> selected this one (-44.7654654219144, -4.588366327941305)
  intersection at (-44.612066720294436, 5.896058254800234), phi=3.01 rad for points [(-44.612066720294436, 5.896058254800234), (0.6512891330810611, 44.9952866694405)]
    checking window 2.36 .. 4.71
  -> selected this one (-44.612066720294436, 5.896058254800234)


In [13]:
def arc_near_emission(C, R, half_span=np.deg2rad(70)):
    # angle of P_emit around center C
    phi_c = math.atan2(P_emit[1]-C[1], P_emit[0]-C[0])
    phis = np.linspace(phi_c - half_span, phi_c + half_span, 400)
    return np.column_stack([C[0] + R*np.cos(phis), C[1] + R*np.sin(phis)])

pre_arc  = arc_near_emission(C0, R0, half_span=np.deg2rad(90))
post_arc = arc_near_emission(C1, R1, half_span=np.deg2rad(90))


In [ ]:
def init():
    ax.cla()
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlim(-50, 0)
    ax.set_ylim(-25, 25)
    ax.set_xlabel("x (cm)")
    ax.set_ylabel("y (cm)")
    return []

def animate(f):
    ax.cla()
    init()
    draw_static()

    ## Phase 1: layers + histograms
    #if f >= 0:
    #    draw_histograms(alpha_scale=min(1.0, 0.3 + 0.1*f))

    # Phase 2: scatter hits
    if f >= 2:
        draw_hits(show_deviant=False)

    # Phase 3: pre-emission fit up to emission
    if f >= 4:
        draw_pre_fit(up_to_P=P_emit)

    # Phase 4: reveal deviating hit on emission layer (cartoon "off-circle")
    if f >= 6:
        draw_hits(show_deviant=True)

    # Phase 5: draw γ from emission point
    if f >= 7:
        draw_gamma()

    # Phase 6: continue with tighter post-emission curvature
    if f >= 8:
        draw_post_fit(from_P=P_emit)

    # Phase 7: EMCal φ extrapolations (pre vs post)
    if f >= 8:
        draw_phi_extrap()

    # Emission marker
    if f >= 5:
        ax.scatter([P_emit[0]],[P_emit[1]], s=55, marker='o', edgecolor='k', facecolor='none')
        ax.text(P_emit[0]+2, P_emit[1]+2, "emission", fontsize=10)

    ax.legend(loc='upper left', framealpha=0.9, fontsize=9, handlelength=2)
    ax.set_title("Bremsstrahlung cartoon: fit before/after emission, γ to EMCal")
    return []


In [17]:
# turn off the widget backend for this cell
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['animation.html'] = 'jshtml'

from IPython.display import HTML

# build your fig/ani here

ani = FuncAnimation(fig, animate, frames=total_frames, interval=700, blit=False, repeat_delay=800)

plt.close(fig)                      # don't also draw the static canvas
HTML(ani.to_jshtml(default_mode='loop'))  # <-- this must be the LAST line and NOT captured


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
